- 문제: https://school.programmers.co.kr/learn/courses/30/lessons/67257

## 풀이1

In [5]:
%%timeit

from itertools import permutations
import re

def get_ops_permutations(expression):
    ops_candidate = ['*', '+', '-'] # 연산자 후보
    ops = [] # 현재 필요한 연산자
    
    for op in ops_candidate:
        if op in expression:
            ops.append(op)
            
    return list(permutations(ops))

def calc(operator, num1, num2):
    if operator == '+':
        return num1 + num2
    elif operator == '-':
        return num1 - num2
    elif operator == '*':
        return num1 * num2
    else:
        raise ValueError('지정된 연산자가 아닙니다.')
    
def solution(expression):
    # 숫자와 연산자 분리
    parts = re.split('(\d+|[-+*])', expression) # 숫자와 연산자를 모두 포함하도록 패턴 작성
    filtered_parts = list(filter(None, parts)) # 필터를 사용하여 빈 문자열을 제거

    op_permutations = get_ops_permutations(expression)
    max_result = 0
    
    for op_list in op_permutations:
        current = filtered_parts

        for op in op_list:
            stack = [current[0]]
            for i in range(1, len(current) - 1, 2):
                if current[i] == op:
                    prev = stack.pop()
                    stack.append(calc(op, int(prev), int(current[i + 1])))
                else:
                    stack.append(current[i])
                    stack.append(current[i + 1])
                    
            current = stack

        max_result = max(max_result, abs(current[0]))
        
    return max_result

solution("100-200*300-500+20") # 60420
solution("50*6-3*2") # 300

42.2 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 풀이: 2 -> stack 이용하기

In [9]:
%%timeit

from itertools import permutations
import re

def get_ops_permutations(expression):
    ops_candidate = ['*', '+', '-'] # 연산자 후보
    ops = [] # 현재 필요한 연산자
    
    for op in ops_candidate:
        if op in expression:
            ops.append(op)
            
    return list(permutations(ops))

def calc(operator, num1, num2):
    if operator == '+':
        return int(num1) + int(num2)
    elif operator == '-':
        return int(num1) - int(num2)
    elif operator == '*':
        return int(num1) * int(num2)
    else:
        raise ValueError('지정된 연산자가 아닙니다.')

def filter_expression(value):
    return int(value) if value.isdigit() else value

def solution(expression):
    # 숫자와 연산자 분리
    parts = re.split('(\d+|[-+*])', expression) # 숫자와 연산자를 모두 포함하도록 패턴 작성
    not_empty_parts = filter(None, parts)
    filtered_parts = list(map(filter_expression, not_empty_parts))  # 필터를 사용하여 빈 문자열을 제거하고 숫자는 정수로 변환

    op_permutations = get_ops_permutations(expression)
    max_result = 0
    
    for op_list in op_permutations:
        current = filtered_parts.copy()

        for op in op_list:
            stack = []
            while current:
                part = current.pop(0)
                if part == op:
                    operand = stack.pop()
                    stack.append(calc(op, operand, current.pop(0)))
                else:
                    stack.append(part)
            current = [str(item) for item in stack]
        
        max_result = max(max_result, abs(int(current[0])))
        
    return max_result

solution("100-200*300-500+20") # 60420
solution("50*6-3*2") # 300

83.5 µs ± 6.43 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## 풀이 3: 시간 복잡도 개선
리스트의 앞부분에서 요소 제거 최적화: list.pop(0) 연산은 리스트의 모든 요소를 한 칸씩 앞으로 이동시켜야 하므로 시간 복잡도가 O(n)입니다. 대안으로, 리스트를 큐로 처리하거나 인덱스를 사용하여 순회하는 방법을 고려할 수 있습니다.

In [10]:
%%timeit

from itertools import permutations
import re

def get_ops_permutations(expression):
    ops_candidate = ['*', '+', '-'] # 연산자 후보
    ops = [] # 현재 필요한 연산자
    
    for op in ops_candidate:
        if op in expression:
            ops.append(op)
            
    return list(permutations(ops))

def calc(operator, num1, num2):
    if operator == '+':
        return int(num1) + int(num2)
    elif operator == '-':
        return int(num1) - int(num2)
    elif operator == '*':
        return int(num1) * int(num2)
    else:
        raise ValueError('지정된 연산자가 아닙니다.')

def filter_expression(value):
    return int(value) if value.isdigit() else value

def solution(expression):
    # 숫자와 연산자 분리
    filtered_parts = list(map(filter_expression, filter(None, re.split('([-+*])', expression))))

    max_result = 0
    for op_list in get_ops_permutations(expression):
        ops = list(op_list)
        nums = filtered_parts[::2]
        operations = filtered_parts[1::2]
                                    
        for op in op_list:
            while op in operations:
                idx = operations.index(op)
                result = calc(op, nums[idx], nums[idx + 1])
                nums.pop(idx + 1)  # 두 번째 피연산자 제거
                nums[idx] = result  # 결과를 첫 번째 피연산자 위치에 저장
                operations.pop(idx)  # 사용된 연산자 제거

        max_result = max(max_result, abs(nums[0]))
        
    return max_result

solution("100-200*300-500+20") # 60420
solution("50*6-3*2") # 300

34 µs ± 869 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
